In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import string
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import LinearSVC


In [ ]:
data_set = pd.read_csv(r'C:\Users\Bluechip\Desktop\DATA ANALYTICS AND TECHNOLOGIES\SECOND SEMESTER\THESIS\data_set.csv')

In [ ]:
data_set.info()

In [ ]:
data_set.describe()

In [ ]:
data_set.shape

In [ ]:
data_set.head(7)

In [ ]:
data_set.isnull().sum()

In [ ]:
dataset1 = data_set.copy()

In [ ]:
#Drop null values 
data_set.dropna(subset=['Class Name','Division Name','Department Name'],inplace=True)


In [ ]:
#Combine Review Text and Title into one
data_set['Reviewed_Text'] = data_set['Title'].fillna('') + '! ' + data_set['Review Text'].fillna('')

#View last 5 rows 
data_set.tail(5)

In [ ]:
data_set.drop(['Title','Review Text','Division Name','Class Name','Positive Feedback Count','Unnamed: 0'],axis=1,inplace=True)

In [ ]:
data_set.info()

In [ ]:
#convert all words into lower case

data_set["Reviewed_Text"] = data_set["Reviewed_Text"].str.lower()


data_set["Reviewed_Text"][18:25]

In [ ]:
#expand contractions
import contractions


def expand_contractions(text):
    return contractions.fix(text)

# Call function on 'Reviews Text' column
data_set["Reviewed_Text"] = data_set["Reviewed_Text"].apply(expand_contractions)

data_set["Reviewed_Text"][18:25]

In [ ]:
#Removing stopwords
stop_words_nltk = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data_set["Reviewed_Text"] = data_set["Reviewed_Text"].apply(lambda text:remove_stopwords(text))

data_set["Reviewed_Text"][30:35]

In [ ]:
#REMOVING PUNCTUATION
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data_set["Reviewed_Text"] = data_set["Reviewed_Text"].apply(lambda text: remove_punctuation(text))

In [ ]:
#REMOVING NUMBERS
data_set["Reviewed_Text"] = data_set["Reviewed_Text"].apply(lambda x : re.sub(r'[^a-z]',' ',x))

data_set["Reviewed_Text"][30:35]

In [ ]:
#LEMMATIZATION

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
data_set["Reviewed_Text"] = data_set["Reviewed_Text"].apply(lambda text: lemmatize_words(text))

data_set["Reviewed_Text"][30:35]

In [ ]:
# Defining a sentiment analyser function for polarity

def sentiment(review):
    return review.apply(lambda Text: pd.Series(TextBlob(Text).sentiment.polarity))

In [ ]:
data_set['polarity'] = sentiment(data_set["Reviewed_Text"])

In [ ]:
def get_sentiment_label(polarity):
    if polarity >= 0.5:
        return 'positve'
    elif polarity < 0.2:
        return 'negative'
    else:
        return 'neutral'

In [ ]:
data_set['Polarity_Sentiment'] = data_set['polarity'].apply(get_sentiment_label)

In [ ]:
data_set.tail(5)

In [ ]:
text = " ".join(i for i in data_set['Reviewed_Text'])
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="black").generate(text)
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


In [ ]:
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def freq_words(x, terms=9):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()
    
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word': list(fdist.keys()), 'count': list(fdist.values())})
    
    # Selecting the top 'terms' frequent words
    d = words_df.nlargest(columns='count', n=terms)
    
    total = d['count'].sum()  # Total count of selected words
    
    plt.figure(figsize=(8, 5))
    ax = sns.barplot(data=d, x="word", y="count")
    ax.set(ylabel="count")
    
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{height}\n({height/total:.1%})',
                    xy=(p.get_x() + p.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)
    
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
freq_words(data_set["Reviewed_Text"])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def n_ngram(corpus,n = None,ngram = 1):
    vec = CountVectorizer(stop_words = 'english',ngram_range=(ngram,ngram)).fit(corpus)
    bag_of_words = vec.transform(corpus) #Have the count of  all the words for each review
    sum_words = bag_of_words.sum(axis =0) #Calculates the count of all the word in the whole review
    words_freq = [(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq,key = lambda x:x[1],reverse = True)
    return words_freq[:n]

In [ ]:
common_words= n_ngram(data_set['Reviews Text'], 10,1)
data = pd.DataFrame(common_words, columns = ['Reviews_Text' , 'count'])
plt.figure(figsize =(10,5))
data.groupby('Reviewed_Text').sum()['count'].sort_values(ascending=False).plot(
kind='bar', title='Top 10 unigrams in review after removing stop words')

total = d['count'].sum()  # Total count of selected words
    
plt.figure(figsize=(8, 5))
  ax = sns.barplot(data=d, x="word", y="count")
    ax.set(ylabel="count")
    
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{height}\n({height/total:.1%})',
                    xy=(p.get_x() + p.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)
    
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
from nltk.util import ngrams
from collections import Counter


# Function to generate n-grams
def n_ngram(text_list, n, min_count):
    all_ngrams = []
    stop_words = set(stopwords.words("english"))
    
    for text in text_list:
        words = [word for word in text.split() if word.lower() not in stop_words]
        ngram_list = list(ngrams(words, n))
        all_ngrams.extend(ngram_list)
    
    ngram_counts = Counter(all_ngrams)
    common_ngrams = [(ngram, count) for ngram, count in ngram_counts.items() if count >= min_count]
    
    return common_ngrams

# Get common unigrams with count
common_words = n_ngram(data_set['Reviewed_Text'], 1, 1)
data = pd.DataFrame(common_words, columns=['Reviewed_Text', 'count'])

# Sort by count in descending order
data = data.sort_values(by='count', ascending=False)

# Plotting the top 10 unigrams
plt.figure(figsize=(10, 5))
ax = sns.barplot(data=data.head(10), x="Reviewed_Text", y="count", palette= 'prism_r')
ax.set(title='Top 10 words using unigrams', xlabel="Words", ylabel="Count")
plt.xticks(rotation=45)

# Annotating with percentages
total = data['count'].sum()
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height}\n',#({height/total:.1%})',
                xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom', fontsize=10)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
def create_wordclouds(text, title): 
  stop_words = list(STOPWORDS)
  wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords = stop_words).generate(str(text))
  plt.figure()
  plt.title(title)
  plt.imshow(wordcloud, interpolation="bilinear")
  plt.axis("off")
  plt.show()

In [ ]:
positive_reviews = data_set['Reviewed_Text'][data_set["Polarity_Sentiment"] == 'positive']
negative_reviews = data_set['Reviewed_Text'][data_set["Polarity_Sentiment"] == 'negative']
neutral_reviews = data_set['Reviewed_Text'][data_set["Polarity_Sentiment"] == 'neutral']

create_wordclouds(positive_reviews, "Positive - Review Wordcloud")
create_wordclouds(negative_reviews, "Negative -  Review Wordcloud")
create_wordclouds(neutral_reviews, "Neutral -  Review Wordcloud")

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")

def get_sentiments(text_review):
    doc = nlp(text_review)
    sentiment = doc._.blob.polarity
    sentiment = round(sentiment, 2)
    
    if sentiment > 0.5:
        label = "POSITIVE"
    elif sentiment < 0.2:
        label = "NEGATIVE"
    else:
        label = "NEUTRAL"
        
    positive_words = []
    negative_words = []
    neutral_words =  []
    adjective_words = []

    for word in doc._.blob.sentiment_assessments.assessments:
        if word[1] > 0:
            positive_words.append(word[0][0])
        elif word[1] < 0:
            negative_words.append(word[0][0])
        else:
            neutral_words.append(word[0][0])

    for token in doc:
        if token.pos_ == "ADJ":
            adjective_words.append(token)

    return label, sentiment, positive_words, negative_words, neutral_words, adjective_words



In [ ]:
data_set.head(5)

In [ ]:
data_set["spacy_analysis"] = [get_sentiments(rev) for rev in data_set["Reviewed_Text"]]
data_set["spacy_sentiment"] = [i[0] for i in data_set["spacy_analysis"]]
data_set["spacy_score"] = [i[1] for i in data_set["spacy_analysis"]]
data_set["spacy_pos"] = [i[2] for i in data_set["spacy_analysis"]]
data_set["spacy_neg"] = [i[3] for i in data_set["spacy_analysis"]]
data_set["spacy_neu"] = [i[3] for i in data_set["spacy_analysis"]]
data_set["spacy_adj"] = [i[4] for i in data_set["spacy_analysis"]]

In [ ]:
positive_reviews = data_set["spacy_pos"]
negative_reviews = data_set["spacy_neg"]
neutral_reviews = data_set["spacy_neu"]

create_wordclouds(positive_reviews, "Positive - Review using Spacy")
create_wordclouds(negative_reviews, "Negative - Review Using Spacy")
create_wordclouds(neutral_reviews, "Neutral - Review Using Spacy")

In [ ]:
age_bins = [18, 26, 36, 46, 56, 66, 76, 86, 96, 99]  # Customize the age groups and bins as per your requirement
age_labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66-75', '76-85', '86-95', '96-99']  # Labels for each age group

# Categorize age groups using pd.cut()
data_set['Age Group'] = pd.cut(data_set['Age'], bins=age_bins, labels=age_labels)

age_group_names = {'18-25': 'Genz','26-35': 'Millennium','36-45': 'GenX','46-55': 
                   'Baby Boomers','56-65': 'Silent Generation','66-75': 'Greatest Generation','76-85': 'Veterans',
                    '86-95': 'Centenarians','96-99': 'Centenarians+'}

data_set['Age Group Name'] = data_set['Age Group'].map(age_group_names)



In [ ]:
 plt.figure(figsize=(10, 5))
ax = sns.countplot(x='Age Group', data=data_set)

total = len(data_set)  # Total number of data points

for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height}\n({height/total:.1%})',
                xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom', fontsize=10)

ax.set_xlabel('Age Group', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_title('Distribution of Age Groups', fontsize=17)
ax.tick_params(labelsize=10)
plt.xticks(rotation=40)

plt.show()


In [ ]:
plt.figure(figsize=(8,8))
ax = sns.countplot(x='Department Name', data=data_set, palette='twilight_shifted')

total = len(data_set)  # Total number of data points

for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height}\n({height/total:.2%})',
                xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom', fontsize=10)

plt.xticks(rotation=45)
plt.title('Purchases in Department ', fontsize=15)
plt.xlabel('Department Name', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()



In [ ]:
#Filter positive reviews and drop rows with missing 'Review Text'
#positive_reviews = data_set[data_set['Polarity_Sentiment'] == 'positive']#.dropna(subset=['Reviewed_Text'])

positive_reviews = data_set[data_set['spacy_sentiment'] == 'NEGATIVE']#.dropna(subset=['Reviewed_Text'])

#positive_reviews = data_set[["spacy_pos", "Reviewed_Text","Age Group"]]

# Create word cloud for different age groups
age_groups = data_set['Age Group'].unique()

# Function to generate word cloud for positive reviews in each age group
def generate_word_cloud_for_age_group(age_group):
    text = " ".join(review for review in positive_reviews[positive_reviews['Age Group'] == age_group]['Reviewed_Text'])
    wordcloud = WordCloud(width=800, height=300, background_color='black').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='kaiser')
    plt.axis('off')
    plt.title(f'Word Cloud for Positive Reviews in Age Group: {age_group}')
    plt.show()
#'antialiased', 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 
#'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman'
# Generate word cloud for each age group
for age_group in age_groups:
    generate_word_cloud_for_age_group(age_group)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
ax = sns.countplot(x='Age Group', hue='Department Name', data=data_set, palette='Set2')

# Calculate count and percentage of each department within each age group
total_counts = data_set.groupby(['Age Group', 'Department Name']).size().unstack()
percentages = total_counts.div(total_counts.sum(axis=1), axis=0) * 100
percentages = percentages.round(0)  # Round to nearest whole number

# Adjust font settings
font = {'size': 17, 'family': 'Franklin Gothic'}
plt.title('Distribution of Purchases by Age Group and Department ', fontsize=20, fontdict=font)
plt.xlabel('Age Group', fontsize=15, fontdict=font)
plt.ylabel('Percentage', fontsize=15, fontdict=font)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Set legend title and show legend
plt.legend(title='Department')

# Display the plot
plt.show()

# Display the count and percentage table
print("Count Table:")
print(total_counts)
print("\nPercentage Table:")
print(percentages)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 10))
ax = sns.countplot(x='Department Name', hue='spacy_sentiment', data=data_set, palette='Set1')

total = len(data_set)  # Total number of data points

for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height}\n({height/total:.1%})',
                xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom', fontsize=10)

plt.title('Sentiment Distribution by Departments', fontsize=20)
plt.xlabel('Departments', fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.xticks(rotation=45)
plt.legend(title='Sentiment')
plt.show()
